# Clean dropbox and mongodb

We have a lot of tests and sometimes simply forget to clean the dropbox. This is a simple code to clean the dropbox.

In [9]:
import sys
from icecream import ic
import dropbox
from dropbox.exceptions import AuthError
from decouple import config

In [2]:
app_key = config("APP_KEY")
app_secret = config("APP_SECRET")
refresh_token = config("REFRESH_TOKEN")

clean the configs

In [12]:
def clean_dummies_from_folder(folder_path:str):
    folder_path = folder_path.strip("/")

    folder_path = "/" + folder_path + "/"
    with dropbox.Dropbox(
        app_key=app_key,
        app_secret=app_secret,
        oauth2_refresh_token=refresh_token,
    ) as dbx:
        # Check that the access token is valid
        try:
            dbx.users_get_current_account()
        except AuthError:
            sys.exit("ERROR: Invalid access token.")

        all_files = [] # collects all files here
        has_more_files = True  # because we haven't queried yet
        cursor = None  # because we haven't queried yet
        while has_more_files:
            if cursor is None: # if it is our first time querying
                folders_results = dbx.files_list_folder(folder_path)
            else:
                folders_results = dbx.files_list_folder_continue(cursor)
            all_files.extend(folders_results.entries)
            cursor = folders_results.cursor
            has_more_files = folders_results.has_more
        for entry in all_files:
            if "dummy" in entry.name:
                print("Deleting folder: " + entry.name)
                full_path = folder_path + entry.name

                print("Deleting folder: " + full_path)
                _ = dbx.files_delete_v2(path=full_path)

In [13]:
backend_config_path = "/Backend_files/Config/"
clean_dummies_from_folder(backend_config_path)


Deleting folder: dummy334fc
Deleting folder: /Backend_files/Config/dummy334fc
Deleting folder: dummy25952
Deleting folder: /Backend_files/Config/dummy25952


Clean the finished jobs

In [15]:
folder_path = "/Backend_files/Finished_Jobs/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Finished_Jobs/dummy24f0f
Deleting folder: dummy4f2ab
Deleting folder: /Backend_files/Finished_Jobs/dummy4f2ab


In [16]:
folder_path = "/Backend_files/Queued_Jobs/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Queued_Jobs/dummy24f0f


In [17]:
folder_path = "/Backend_files/Result/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Result/dummy24f0f


In [18]:
folder_path = "/Backend_files/Status/"
clean_dummies_from_folder(folder_path)

Deleting folder: dummy24f0f
Deleting folder: /Backend_files/Status/dummy24f0f


## Mongodb

For the mongodb we have to do similiar things.

In [1]:
from pymongo.mongo_client import MongoClient
from bson.objectid import ObjectId
from bson.errors import InvalidId

In [4]:
mongodb_username = config("MONGODB_USERNAME")
mongodb_password = config("MONGODB_PASSWORD")
mongodb_database_url = config("MONGODB_DATABASE_URL")
uri = f"mongodb+srv://{mongodb_username}:{mongodb_password}@{mongodb_database_url}"
uri = uri + "/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client: MongoClient = MongoClient(uri)

In [5]:
database = client["backends"]
collection = database["configs"]
# get all the documents in the collection configs and save the disply_name in a list

backend_names: list = []
for config_dict in collection.find():
    if "display_name" in config_dict:
        if "dummy" in config_dict["display_name"]:
            print("Deleting config: " + config_dict["display_name"])
            collection.delete_one({"_id": ObjectId(config_dict["_id"])})
    if "payload" in config_dict:
        if "display_name" in config_dict["payload"]:
            print(config_dict["payload"])
            if "dummy" in config_dict["payload"]["display_name"]:
                print("Deleting config: " + config_dict["payload"]["display_name"])
                collection.delete_one({"_id": ObjectId(config_dict["_id"])})
        else:
            print("Deleting config: " + config_dict["display_name"])
            collection.delete_one({"_id": ObjectId(config_dict["_id"])})
    else:
        print("Deleting random config")
        collection.delete_one({"_id": ObjectId(config_dict["_id"])})
        

{'description': 'Setup of a cold atomic gas experiment with a single qudit.', 'version': '0.3', 'display_name': 'singlequdit', 'cold_atom_type': 'spin', 'gates': [{'coupling_map': [[0]], 'description': 'Evolution under Lx', 'name': 'rlx', 'parameters': ['omega'], 'qasm_def': 'gate lrx(omega) {}'}, {'coupling_map': [[0]], 'description': 'Evolution under the Z gate', 'name': 'rlz', 'parameters': ['delta'], 'qasm_def': 'gate rlz(delta) {}'}, {'coupling_map': [[0]], 'description': 'Evolution under lz2', 'name': 'rlz2', 'parameters': ['chi'], 'qasm_def': 'gate rlz2(chi) {}'}, {'coupling_map': [[0]], 'description': 'Apply the full time evolution on the array.', 'name': 'sq_full', 'parameters': ['omega, delta, chi'], 'qasm_def': 'gate sq_full(omega, delta, chi) {}'}], 'max_experiments': 1000, 'max_shots': 1000000, 'simulator': True, 'supported_instructions': ['rlx', 'rlz', 'rlz2', 'barrier', 'measure', 'load', 'sq_full'], 'num_wires': 1, 'wire_order': 'interleaved', 'num_species': 1, 'pending

Now also clean out stupid collections of finished jobs.

In [8]:
def clean_dummies_from_database(database_name: str):
    """
    This function deletes all collections in the database that have the word dummy in them
    """
    database = client[database_name]

    # now get a list of all existing collections within the database
    collections = database.list_collection_names()
    print(collections)
    for name in collections:
        if "dummy" in name:
            print("Deleting collection: " + name)
            collection = database[name]
            collection.drop()

In [9]:
clean_dummies_from_database("jobs")

['queued.fermions', 'deleted', 'running', 'finished.fermions']


In [10]:
clean_dummies_from_database("results")

['fermions', 'rydberg']


In [11]:
clean_dummies_from_database("status")

['dummyf7c34', 'dummy1e817', 'dummy2a868', 'dummy7391d', 'dummy86987', 'dummy0cd98', 'dummy7f2c2', 'dummye4771', 'dummye8ec1', 'dummy7485d', 'dummy31b09', 'dummy86239', 'dummy0d3ce', 'dummybc8f1', 'dummy3cef2', 'dummy44bd0', 'dummy94249', 'dummyc888d', 'dummy60718', 'dummy845e6', 'dummy311bc', 'dummye9b18', 'dummy21ac7', 'dummy8fd4b', 'dummybaed7', 'dummy3a66f', 'dummy20b46', 'dummycaaeb', 'dummy62910', 'dummy6b277', 'dummy75362', 'dummy3a1b8', 'dummy75d65', 'dummy20459', 'dummy32227', 'dummy0f2a0', 'dummycf4a9', 'dummy32737', 'dummy67ab5', 'dummybbef8', 'dummya67d3', 'dummy4043d', 'dummy7c6b0', 'dummy19aba', 'dummy342df', 'dummy07eee', 'dummy916b7', 'dummye94c2', 'dummy0f898', 'dummy02c1d', 'dummy54164', 'dummyccf04', 'dummyb4d82', 'dummy1192a', 'dummy39dbd', 'dummy20f76', 'dummy4bbcd', 'dummy868eb', 'dummy02b40', 'dummy9f8e2', 'dummya0f27', 'dummy04cd0', 'dummy8e76a', 'dummyf8b49', 'dummy52ec5', 'dummy46fad', 'dummyb76d3', 'dummycf0ff', 'dummyf1d6b', 'dummy08a93', 'dummya3319', 'dumm